In [1]:
#importing basic lib and loading train data
import pandas 
import numpy 
train_data = pandas.read_csv("train.csv")
test_data = pandas.read_csv("test.csv")
validate_data = pandas.read_csv("valid.csv")

In [2]:
pandas.options.display.max_colwidth = 2000

In [3]:
# looking into the data
lst1 = train_data.head(1)
lst2 = test_data.head(1)
lst3 = validate_data.head(1)
print(lst1)
print(lst2)
print(lst3)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         text  \
0  I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" b

#### now we know that the data has all sort of words with lowecase uppercase words punctuators everthing so we will handle all that

In [4]:
pandas.options.display.max_colwidth = 50

In [5]:
train_data.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [6]:
print(train_data.shape)
print(test_data.shape)
print(validate_data.shape)

(40000, 2)
(5000, 2)
(5000, 2)


In [7]:
train_data['label'].value_counts()

0    20019
1    19981
Name: label, dtype: int64

In [8]:
majority= train_data[train_data['label']==0]
minority= train_data[train_data['label']==1]
reduced_majority = majority.sample(len(minority))
balance_data = pandas.concat([reduced_majority,minority])
balance_data = balance_data.sample(frac=1).reset_index(drop = True)
balance_data.to_csv('undersampled_train.csv',index=False)
reloaded_data_train = pandas.read_csv('undersampled_train.csv')

In [9]:
reloaded_data_train['label'].value_counts()

1    19981
0    19981
Name: label, dtype: int64

In [10]:
reloaded_data_train['text'][0]

'Twenty five years ago, I showed this film in some children\'s classes in Entomology and can still remember the excitement of the kids; they were spellbound! It is not just about the termites who have built and live in the "Castles of Clay," but also about the other animals who use the mounds. There is a fantastic scene in which a cobra fights a monitor lizard while a colony of mongooses watch. It is a not only good for entomology classes, but also for teaching about ecology since there is so much about the interactions between the termites and other organisms and the whole ecology of all of the organisms that live in and around the mounds. <br /><br />I wish it was available on DVD, so that I could watch it again and show others.'

In [11]:
# first we will preprocess data and remove all the punctuators
import string
from nltk.corpus import stopwords
punctuators = list(string.punctuation)
stop = stopwords.words('english')
all_stop = stop + punctuators 
filtered_sentence_train = []
for sentence in reloaded_data_train['text']:
    words = sentence.split()
    filtered_words = [word for word in words if word.lower() not in all_stop]
    filtered_sentence_train.append(' '.join(filtered_words))

In [12]:
# as the data is still not clean
import re
def clean_word(word):
    return re.sub(r'\W+$', '', word)

In [13]:
from nltk.tokenize import sent_tokenize, word_tokenize
filtered_sentence_train = []
for sentence in reloaded_data_train['text']:
    sentence =  re.sub(r'\bbr\b', '', sentence)
    words = word_tokenize(sentence)
    cleaned_words = [clean_word(word) for word in words] 
    filtered_words = [word for word in cleaned_words if word.lower() not in all_stop]
    filtered_sentence_train.append(' '.join(filtered_words))

In [14]:
#lookint at the data now
filtered_sentence_train[0:3]

["Twenty five years ago  showed film children 's classes Entomology still remember excitement kids  spellbound  termites built live  Castles Clay   also animals use mounds  fantastic scene cobra fights monitor lizard colony mongooses watch  good entomology classes  also teaching ecology since much interactions termites organisms whole ecology organisms live around mounds        wish available DVD  could watch show others ",
 "really liked film.it features John Wayne first starring performance.even  tell Wayne real presence  although would n't really mature icon known Stagecoach,9 years later 1939.it 's settlers country heading new west colonize it.Wayne 's character Breck Coleman joins  personal reasons.most main actors stage actors never done film  makes movie even amazing.they managed create believable  distinctive characters quite oddball mix here.Cimarron would come year later  similar story  though n't like much movie.for  Big Trail strong 8/10 ",
 "found film bit depressing  n't 

In [15]:
#loading and preprocessing validation file we can also create a function for this but here ill do directly
validate_data['label'].value_counts()

1    2514
0    2486
Name: label, dtype: int64

In [16]:
#we can create a function but ill do directly
majority= validate_data[validate_data['label']==1]
minority= validate_data[validate_data['label']==0]
reduced_majority = majority.sample(len(minority))
balance_data = pandas.concat([reduced_majority,minority])
balance_data = balance_data.sample(frac=1).reset_index(drop = True)
balance_data.to_csv('undersampled_validate.csv',index=False)
reloaded_data_valid = pandas.read_csv('undersampled_validate.csv')

In [17]:
#now data is fine
reloaded_data_valid['label'].value_counts()

1    2486
0    2486
Name: label, dtype: int64

In [18]:
filtered_sentence_valid= []
for sentence in reloaded_data_valid['text']:
    sentence =  re.sub(r'\bbr\b', '', sentence)
    words = word_tokenize(sentence)
    cleaned_words = [clean_word(word) for word in words] 
    filtered_words = [word for word in cleaned_words if word.lower() not in all_stop]
    filtered_sentence_valid.append(' '.join(filtered_words))

In [19]:
#the data is consistent now
print(len(reloaded_data_train['label']))
print(len(filtered_sentence_train))
print(len(reloaded_data_valid['label']))
print(len(filtered_sentence_valid))

39962
39962
4972
4972


In [20]:
#oading the undersampled data and now we will implement svm on the filtered data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
vectorizer = TfidfVectorizer()
X_train =  vectorizer.fit_transform(filtered_sentence_train)
Y_train = reloaded_data_train['label']

In [21]:
X_valid = vectorizer.transform(filtered_sentence_valid)
Y_valid = reloaded_data_valid['label']

In [ ]:
#training the svm model
svm_model = SVC()
svm_model.fit(X_train, Y_train)
Y_pred = svm_model.predict(X_valid)
# Evaluating the model
accuracy = accuracy_score(Y_valid, Y_pred)
report = classification_report(Y_valid, Y_pred)
print("Accuracy:", accuracy)
print("Classification Report:",report)